In [1]:
import sys
import time
import tensorflow as tf
import numpy as np
sys.path.insert(0, '../')
%load_ext autoreload
%autoreload 2

In [2]:
from seqmodel.bunch import Bunch
from seqmodel import model
from seqmodel import data

In [3]:
tf.reset_default_graph()
model_opt = model.seq_model.BasicSeqModel.default_opt()
model_fn = model.seq_model.BasicSeqModel(model_opt)
lm = model_fn(is_training=True)
optimizer = tf.train.AdamOptimizer()
g_v_pairs =  optimizer.compute_gradients(lm.training_loss, tf.trainable_variables())
train_op = optimizer.apply_gradients(g_v_pairs)

In [4]:
vocab = data.Vocabulary.from_vocab_file('../data/tiny_single/vocab.txt')
valid_iter = data.TokenIterator(vocab, vocab)
valid_iter.initialize('../data/tiny_single/valid.txt')
valid_iter.init_batch(4)

In [5]:
sess_config = tf.ConfigProto(device_count={'GPU': 0})
sess = tf.Session(config = sess_config)
sess.run(tf.global_variables_initializer())
batch = valid_iter.next_batch()
state = None

In [6]:
prediction, state, info = lm.predict(sess, batch.features, state=state, new_seq=True)

In [7]:
eval_loss, state, info = lm.evaluate(sess, batch)

In [8]:
eval_loss, train_loss, state, info = lm.train(sess, batch, train_op)

In [9]:
print(lm._nodes.to_pretty())

_all_:
  decode:
    decode_output:
      distribution: Tensor("model/rnn_decoder_1/decoder_rnn/Reshape_3:0", dtype=float32)
      final_state: LSTMStateTuple(c=<tf.Tensor 'model/rnn_decoder_1/decoder_rnn/rnn/while/Exit_2:0' shape=(?, 128) dtype=float32>, h=<tf.Tensor 'model/rnn_decoder_1/decoder_rnn/rnn/while/Exit_3:0' shape=(?, 128) dtype=float32>)
      initial_state: LSTMStateTuple(c=<tf.Tensor 'model/rnn_decoder_1/decoder_rnn/zeros:0' shape=(?, 128) dtype=float32>, h=<tf.Tensor 'model/rnn_decoder_1/decoder_rnn/zeros_1:0' shape=(?, 128) dtype=float32>)
      logit: Tensor("model/rnn_decoder_1/decoder_rnn/div:0", dtype=float32)
      logit_temperature: Tensor("model/rnn_decoder_1/decoder_rnn/logit_temperature:0", dtype=float32)
      rnn:
        _logit_b: Tensor("model/decoder_rnn/logit_b/read:0", shape=(15,), dtype=float32)
        _logit_w: Tensor("model/decoder_rnn/logit_w/read:0", shape=(15, 128), dtype=float32)
        cell_output: Tensor("model/rnn_decoder_1/decoder_rnn/rnn/T